In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# load libraries
from FRVRS import nu, fu
from numpy import nan
from pandas import DataFrame, read_csv, read_excel, concat, get_dummies, isna
from re import split, search, sub, MULTILINE
from scipy.stats import f_oneway, ttest_ind, kruskal, norm
import itertools
import os.path as osp
import re
import statsmodels.api as sm

In [9]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_csv_stats_df=''
)
csv_stats_df = data_frames_dict['metrics_evaluation_open_world_csv_stats_df'].copy()
print(sorted([cn for cn in csv_stats_df.columns if 'all' in cn]))

No pickle exists for metrics_evaluation_open_world_csv_stats_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_csv_stats_df.csv.
[]


In [ ]:

Fearch for teleports and get the euclidean distance to the nearest patient.

In [15]:

mask_series = (csv_stats_df.injury_required_procedure == 'none')
';'.join(csv_stats_df[mask_series].csv_file_name.unique().tolist())

'02d472ac-f6fe-474c-815d-6125fadfcbf7_2024211.csv;04f80090-9e61-431d-8473-dccb75fed04d_2024207.csv;05e0be07-479a-4ea4-9b90-3fc7b79ef7ff_2024225.csv;0bc62b93-ac4d-40ef-bdfe-1d7badb24f70_2024208.csv;0e8693a1-1926-4d56-b0c4-ee2dc8181fc6_2024226.csv;1995e7ef-ef02-4fc1-b1ab-f137dbf69d48_2024217.csv;21f8cb5d-f5ac-4a01-9287-43df5f6751a1_2024219.csv;220b609b-0e35-454e-9afd-c84cbfa3e3ad_2024202.csv;23081f6e-875e-44f5-8bd0-edc3905f5c2c_2024201.csv;287389c4-4c48-4483-87c0-6b363b57bde2_2024220.csv;37a554ee-fc49-4730-819c-2d97727bb0b7_2024218.csv;385032e9-9801-4dcf-a841-b3703a0d9acd_2024209.csv;3cf14e31-f416-4c78-8a69-91bf0c685448_2024215.csv;45365e18-6e38-48e7-b4a2-6b448b209034_2024218.csv;499179ba-3138-4bae-918e-ffc7fb943760_2024206.csv;4bc46c8c-66e7-463d-b3a1-2a8303af4fd1_2024203.csv;50b15e40-9860-4574-8ab8-0bd960fe27de_2024220.csv;5a909dc4-0f86-430d-a400-e37139cba69f_2024212.csv;5d8d73a3-1898-4f64-8676-73edd1b7daa0_2024211.csv;67dc0230-511d-41ac-ae9b-850900ab9e6a_2024208.csv;6db9446c-2cd4-41b4-

In [7]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_anova_df=''
)
anova_df = data_frames_dict['metrics_evaluation_open_world_anova_df'].copy()
print(sorted([cn for cn in anova_df.columns if 'trust' in cn.lower()]))

No pickle exists for metrics_evaluation_open_world_anova_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_anova_df.csv.
['mean_PropTrust']


In [8]:

anova_df.mean_PropTrust.unique().tolist()

[3.666666667, 4.333333333, 4.0, 3.333333333, 2.333333333, 0.0]

In [4]:

def entitle_column_name(column_name):
    """
    Entitles a column name based on specified rules.
    
    Parameters:
        column_name (str):
            The name of the column to be entitled.
    
    Returns:
        str
            The entitled column name.
    
    Notes:
        - If the column name starts with 'mean_' and the substring after 'mean_' exists in the value_description_dict,
          the entitled name is obtained from the corresponding value in the dictionary, prepended with 'Average ' if necessary.
        - Otherwise, the column name is split into parts, and each part is processed to ensure proper capitalization and word replacements.
    """
    
    # Check if the column name starts with 'mean_' and is present in the value_description_dict
    if column_name.startswith('mean_') and (column_name[5:] in value_description_dict):
        entitled_name = value_description_dict[column_name[5:]]
        
        # Prepend 'Average ' if the entitled name does not already start with it
        if not entitled_name.startswith('Average '):
            entitled_name = 'Average ' + entitled_name
    else:
        new_parts_list = []
        old_parts_list = [op for op in split('_', column_name, 0) if op]  # Split the column name by underscores
        for name_part in old_parts_list:
            
            # Check if the part contains a capital letter followed by lowercase letters (camelCase)
            if search('[A-Z][a-z]+', name_part):
                humps_list = [hp for hp in split('([A-Z][a-z]+)', name_part, 0) if hp]  # Split camelCase parts
                for i, hump_part in enumerate(humps_list):
                    
                    # Capitalize each part if it is all lowercase
                    if hump_part == hump_part.lower():
                        humps_list[i] = hump_part.title()
                    
                    # Replace specific abbreviations with full words
                    elif hump_part == 'Sim':
                        humps_list[i] = 'Simulation'
                    elif hump_part == 'Yrs':
                        humps_list[i] = 'Years of'
                    elif hump_part == 'Mil':
                        humps_list[i] = 'Military'
                    elif hump_part == 'Exp':
                        humps_list[i] = 'Experience'
                new_parts_list.extend(humps_list)
            else:
                
                # Process parts that are not in camelCase
                if name_part == name_part.lower():
                    
                    # Capitalize if the part is all lowercase and meets certain conditions
                    if (len(name_part) > 2) and (name_part != 'uuid'):
                        name_part = name_part.title()
                    
                    # Convert certain parts to uppercase
                    elif name_part not in ['to', 'of', 'per']:
                        name_part = name_part.upper()
                new_parts_list.append(name_part)
        
        # Replace 'Mean' with 'Average' if it is the first part
        if new_parts_list[0] == 'Mean':
            new_parts_list[0] = 'Average'
        entitled_name = ' '.join(new_parts_list)
    
    return entitled_name

In [5]:

print("\nGet column and value descriptions")
file_path = osp.join(nu.data_folder, 'xlsx', 'Metrics_Evaluation_Dataset_organization_for_BBAI.xlsx')
dataset_organization_df = read_excel(file_path)

print("\nFix the doubled up descriptions")
mask_series = dataset_organization_df.Labels.map(lambda x: ';' in str(x))
for row_index, label in dataset_organization_df[mask_series].Labels.items():
    labels_list = split(' *; *', str(label), 0)
    dataset_organization_df.loc[row_index, 'Labels'] = labels_list[0]
    
    # Append the new row to the DataFrame
    new_row = dataset_organization_df.loc[row_index].copy()
    new_row['Labels'] = labels_list[1]
    dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

# Append the AD_Del_Omni_Text row to the DataFrame
mask_series = (dataset_organization_df.Variable == 'AD_Del_Omni')
new_row = dataset_organization_df.loc[mask_series].copy()
new_row['Variable'] = 'AD_Del_Omni_Text'
dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

print("\nGet the column value descriptions")
mask_series = ~dataset_organization_df.Description.isnull()
df = dataset_organization_df[mask_series]
value_description_dict = df.set_index('Variable').Description.to_dict()
new_description_dict = value_description_dict.copy()
for k, v in value_description_dict.items():
    new_description_dict[k] = v
    if (not k.endswith('_Text')):
        new_key_name = f'{k}_Text'
        new_description_dict[new_key_name] = new_description_dict.get(new_key_name, v)
value_description_dict = new_description_dict.copy()


Get column and value descriptions

Fix the doubled up descriptions

Get the column value descriptions


In [6]:

import inspect

print('\nWrite up the steps to do the ANOVA stats columns calculations')
comment_regex = re.compile('^ *# ([^\r\n]+)', MULTILINE)
function_call_dict = {
    'encounter_layout': 'fu.add_encounter_layout_column', 'medical_role': 'fu.add_medical_role_column',
    'mean_prioritize_high_injury_severity_patients': 'fu.add_prioritize_severity_column'
}
survey_columns = ['AD_KDMA_Sim', 'AD_KDMA_Text', 'PropTrust', 'ST_KDMA_Sim', 'ST_KDMA_Text', 'YrsMilExp']
mean_survey_columns = ['mean_' + cn for cn in survey_columns]
engaged_patient_columns_list = []
function_calls_list = []
for cn in anova_df.columns:
    if 'engaged_patient' in cn:
        engaged_patient_columns_list.append(cn)
        continue
    print('')
    print(f'{cn} ({entitle_column_name(cn)})')
    print('Steps Needed to do Calculations:')
    comments_list = []
    if cn in ['participant_id', 'scene_id', 'session_uuid']:
        if cn == 'scene_id':
            comments_list.append('The scene_id is derived from the CSV SESSION_START and SESSION_END entries.')
        else:
            comments_list.append(
                f'The {cn} is found in both the CSV and the JSON data in the'
                + ' "Human_Sim_Metrics_Data 4-12-2024.zip" file provided by CACI.'
            )
    else:
        comments_list.append('Group your dataset by participant_id, session_uuid, and scene_id.')
        if cn in mean_survey_columns:
            comments_list.extend([
                f'Find the {cn.replace("mean_", "")} column in the participant_data_0420 spreadsheet provided by CACI for that participant',
                f'The {cn.replace("mean_", "")} value is semi-continously numeric, and you can average it for whatever grouping you need'
            ])
        else:
            if cn in function_call_dict:
                function_call = function_call_dict[cn]
            else:
                function_call = cn.replace('mean_', 'fu.get_')
            function_calls_list.append(function_call)
            source_code = inspect.getsource(eval(function_call))
            comments_list.extend([comment_str for comment_str in comment_regex.findall(source_code) if comment_str and ('verbose' not in comment_str)])
    for i, comment_str in enumerate(comments_list):
        print(f'{i+1}. {comment_str}.')
if engaged_patient_columns_list:
    print('Split the patients metadata ({nu.conjunctify_nounts(engaged_patient_columns_list)}) from the patients list')
print(function_calls_list)


Write up the steps to do the ANOVA stats columns calculations

participant_id (Participant ID)
Steps Needed to do Calculations:
1. The participant_id is found in both the CSV and the JSON data in the "Human_Sim_Metrics_Data 4-12-2024.zip" file provided by CACI..

scene_id (Scene ID)
Steps Needed to do Calculations:
1. The scene_id is derived from the CSV SESSION_START and SESSION_END entries..

session_uuid (Session UUID)
Steps Needed to do Calculations:
1. The session_uuid is found in both the CSV and the JSON data in the "Human_Sim_Metrics_Data 4-12-2024.zip" file provided by CACI..

mean_AD_KDMA_Sim (Average KDMA measurement from simulator probe responses)
Steps Needed to do Calculations:
1. Group your dataset by participant_id, session_uuid, and scene_id..
2. Find the AD_KDMA_Sim column in the participant_data_0420 spreadsheet provided by CACI for that participant.
3. The AD_KDMA_Sim value is semi-continously numeric, and you can average it for whatever grouping you need.

mean_AD

In [27]:

cn, function_calls_list[-1]

('mean_prioritize_high_injury_severity_patients', 'fu.get_prioritize_high_injury_severity_patients')

In [4]:

# load data frames
data_frames_dict = nu.load_data_frames(
    verbose=True, metrics_evaluation_open_world_scene_stats_df=''
)
scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df'].copy()
print(sorted([cn for cn in scene_stats_df.columns if 'all' in cn]))

No pickle exists for metrics_evaluation_open_world_scene_stats_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_scene_stats_df.csv.
['all_patient_injuries_count']


In [5]:

groupby_columns = ['participant_id', 'session_uuid', 'scene_id']
rows_list = []
for (participant_id, session_uuid, scene_id), scene_df in scene_stats_df.groupby(groupby_columns):
    row_dict = {}
    for cn in groupby_columns: row_dict[cn] = eval(cn)
    for cn in scene_df.columns:
        if 'injur' in cn:
            row_dict[cn] = scene_df[cn].squeeze()
    rows_list.append(row_dict)
injury_counts_df = DataFrame(rows_list)

In [6]:

columns_list = [cn for cn in injury_counts_df.columns if 'injur' in cn]
injury_counts_df[columns_list]

,all_patient_injuries_count,injury_correctly_treated_count,injury_not_treated_count,injury_wrongly_treated_count
0,31,15,10,6
1,18,0,18,0
2,0,0,0,0
3,0,0,0,0
4,23,11,8,4
...,...,...,...,...
63,19,5,9,5
64,23,12,7,4
65,31,23,2,6
66,31,10,20,1


In [11]:

assert (
    scene_stats_df.all_patient_injuries_count == scene_stats_df[
        ['injury_correctly_treated_count', 'injury_not_treated_count', 'injury_wrongly_treated_count']
    ].sum(axis='columns')
).all(), "The sum of the various injury counts does not match the total injury count for all rows."

In [8]:

df = injury_counts_df.sample(5)
print(f"""
        # Create a mock DataFrame with all injury columns
        self.injury_counts_df = DataFrame({{
            "all_patient_injuries_count": {df.all_patient_injuries_count.tolist()}
            "injury_correctly_treated_count": {df.injury_correctly_treated_count.tolist()}
            "injury_not_treated_count": {df.injury_not_treated_count.tolist()}
            "injury_wrongly_treated_count": {df.injury_wrongly_treated_count.tolist()}
        }})""")


        # Create a mock DataFrame with all injury columns
        self.injury_counts_df = DataFrame({
            "all_patient_injuries_count": [20, 32, 25, 0, 18]
            "injury_correctly_treated_count": [3, 19, 9, 0, 0]
            "injury_not_treated_count": [13, 11, 11, 0, 18]
            "injury_wrongly_treated_count": [4, 2, 5, 0, 0]
        })


In [22]:

groupby_columns = ['participant_id', 'session_uuid', 'scene_id']
millisecond_threshold = 0
for (participant_id, session_uuid, scene_id), scene_df in csv_stats_df.groupby(groupby_columns):
    
    # Loop through each injury and make a determination if it's treated or not
    injury_correctly_treated_count = 0
    for patient_id, patient_df in scene_df.groupby('patient_id'):
        for injury_id, injury_df in patient_df.groupby('injury_id'):
            is_correctly_treated = fu.get_is_injury_correctly_treated(injury_df, patient_df)
            if is_correctly_treated: injury_correctly_treated_count += 1

In [23]:

mask_series = (injury_df.action_type == 'INJURY_TREATED')
action_ticks_list = sorted(injury_df[mask_series].action_tick.unique())
mask_series = ~injury_df.injury_required_procedure.isnull()
required_procedure = injury_df[mask_series].injury_required_procedure.mode().squeeze()
print(required_procedure)
for action_tick in action_ticks_list:
    mask_series = patient_df.action_tick.map(
        lambda ts: abs(ts - action_tick) < millisecond_threshold
    ) & patient_df.action_type.isin(['TOOL_APPLIED'])
    if mask_series.any():
        print([(required_procedure == self.tool_type_to_required_procedure_dict.get(tool_type)) for tool_type in patient_df[mask_series].tool_applied_type])
        is_correctly_treated = any(
            [(required_procedure == self.tool_type_to_required_procedure_dict.get(tool_type)) for tool_type in patient_df[mask_series].tool_applied_type]
        )

Series([], dtype: object)


In [24]:

injury_df.dropna(axis='columns', how='all').T

,139341
action_type,INJURY_RECORD
action_tick,19200
event_time,2024-04-05 12:10:51
session_uuid,9f3bb117-3a4b-4bc7-9237-e2d9920d5333
csv_file_subpath,Human_Sim_Metrics_Data_4-12-2024/9f3bb117-3a4b...
injury_record_id,L Shoulder Broken
injury_record_patient_id,Patient X Root
injury_record_required_procedure,splint
injury_record_severity,medium
injury_record_body_region,leftArm


In [16]:

columns_list = ['action_type', 'action_tick']
patient_df[columns_list]

,action_type,action_tick
42381,PATIENT_ENGAGED,1262495
42407,BREATHING_CHECKED,1266435
42408,INJURY_TREATED,1267526
42409,TOOL_APPLIED,1267526
42420,INJURY_TREATED,1273515
42422,PULSE_TAKEN,1273567
42483,INJURY_TREATED,1281386
42485,PULSE_TAKEN,1281897
42508,INJURY_TREATED,1286988
42626,PATIENT_DEMOTED,192305


In [29]:

from pandas import Series

groupby_columns = ['participant_id', 'session_uuid', 'scene_id', 'patient_id']
for (participant_id, session_uuid, scene_id, patient_id), patient_df in csv_stats_df.groupby(groupby_columns):
    for injury_id, injury_df in patient_df.groupby('injury_id'):
        action_ticks_list = sorted(injury_df.action_tick.unique())
        mask_series = ~patient_df.tool_type.isnull() & patient_df.action_tick.isin(action_ticks_list)
        if mask_series.any():
            df = patient_df[mask_series]
            mask_series = ~injury_df.injury_record_required_procedure.isnull()
            required_procedure = injury_df[mask_series].injury_record_required_procedure.mode().squeeze()
            if isinstance(required_procedure, Series):
                required_procedure = '(who knows?)'
                raise
            for tool_type in df.tool_type:
                is_correctly_treated = (required_procedure == fu.tool_type_to_required_procedure_dict.get(tool_type))
                print(
                    f'{patient_id} has a {injury_id} which requires a {required_procedure} and it {"was" if is_correctly_treated else "was not"} correctly treated'
                )
                raise

Adept Shooter Root has a R Bicep Puncture which requires a tourniquet and it was correctly treated


RuntimeError: No active exception to reraise

In [30]:

columns_list = [cn for cn in injury_df.columns if 'required_procedure' in cn]
injury_df[columns_list]

,injury_record_required_procedure,injury_treated_required_procedure,injury_required_procedure
42658,tourniquet,NaN,tourniquet
44199,NaN,tourniquet,tourniquet


In [54]:

osp.basename(osp.join('../data/logs', df.csv_file_subpath.squeeze()))

'04f80090-9e61-431d-8473-dccb75fed04d_2024207.csv'